In [1]:
import os
import sys
PROJECT_PATH = "/home/albert/Baikal-ML/" #insert your project path
sys.path.append(f"{PROJECT_PATH}")
from time import time

import numpy as np
import uproot as ur
import polars as pl
import pandas as pd
import pyarrow.parquet as pq
import pyarrow.dataset as pyarr_ds

from data.root_manager.root_reader import RootReader
from data.data_processor import Processor

In [2]:
prty_type = 'muatm'
#path_to_roots = f'{PROJECT_PATH}data/mock_MC_2020/{prty_type}/root/all/'
path_to_roots = f'/net/62/home3/ivkhar/Baikal/data/initial_data/MC_2020/muatm/root/all/'
#path_to_roots = '/home/albert/Baikal-ML/data/mock_MC_2020/muatm/root/all/'

files = os.listdir(f'{path_to_roots}')[0:100]
paths = sorted([f"{path_to_roots}{file}" for file in files])
paths[0:5]

['/net/62/home3/ivkhar/Baikal/data/initial_data/MC_2020/muatm/root/all/10170.root',
 '/net/62/home3/ivkhar/Baikal/data/initial_data/MC_2020/muatm/root/all/1021.root',
 '/net/62/home3/ivkhar/Baikal/data/initial_data/MC_2020/muatm/root/all/10436.root',
 '/net/62/home3/ivkhar/Baikal/data/initial_data/MC_2020/muatm/root/all/10475.root',
 '/net/62/home3/ivkhar/Baikal/data/initial_data/MC_2020/muatm/root/all/10558.root']

In [3]:
proc = Processor(paths)

# Read data from root

In [5]:
paths_slice = paths[0:5]
dfs = proc.read_chunk_as_dfs(paths_slice, proc._extract_prefixes(paths_slice))
df = proc.enrich_and_filter(*dfs)

In [20]:
# test parquet format
dts = []
for i, path_num in enumerate([5,10,15,20, 25]):
    t0 = time()
    paths_slice = paths[0:path_num]
    dfs = proc.read_chunk_as_dfs(paths_slice, proc._extract_prefixes(paths_slice))
    df = proc.enrich_and_filter(*dfs)
    #df.write_parquet('./test.parquet')
    #pyarr_ds.write_dataset(df.to_arrow(), './test_ds/', format='parquet', existing_data_behavior='overwrite_or_ignore')
    #df.write_parquet('./test.parquet')
    print(f"{path_num=}, loading_time per file: {(time()-t0)/path_num:.4f} s")

path_num=5, loading_time per file: 1.6145 s
path_num=10, loading_time per file: 1.4978 s
path_num=15, loading_time per file: 1.3524 s
path_num=20, loading_time per file: 1.2557 s
path_num=25, loading_time per file: 1.2605 s


In [10]:
# test hdf5 format
dts = []
for i, path_num in enumerate([5,10,15,20, 25]):
    t0 = time()
    paths_slice = paths[0:path_num]
    dfs = proc.read_chunk_as_dfs(paths_slice, proc._extract_prefixes(paths_slice))
    df = proc.enrich_and_filter(*dfs)
    df = df.to_pandas()
    df.to_hdf('test.h5', key='data', mode='w', complevel=1)
    #pyarr_ds.write_dataset(df.to_arrow(), './test_ds/', format='parquet', existing_data_behavior='overwrite_or_ignore')
    #df.write_parquet('./test.parquet')
    print(f"{path_num=}, loading_time per file: {(time()-t0)/path_num:.4f} s")

/tmp/ipykernel_3039287/2885806558.py:9: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['ev_id', 'PulsesChID', 'PulsesAmpl', 'PulsesTime', 'PulsesFlg',
       'is_signal', 'X', 'Y', 'Z', 'Xrel', 'Yrel', 'Zrel', 'RespMuTheta',
       'RespMuPhi', 'RespMuTrackX', 'RespMuTrackY', 'RespMuTrackZ',
       'RespMuDelay', 'RespMuEn'],
      dtype='object')]

  df.to_hdf('test.h5', key='data', mode='w', complevel=1)


path_num=5, loading_time per file: 3.6486 s
path_num=10, loading_time per file: 2.5984 s
path_num=15, loading_time per file: 2.4899 s
path_num=20, loading_time per file: 2.4794 s
path_num=25, loading_time per file: 2.2836 s


In [6]:
df

PulsesN,PrimeTheta,PrimePhi,PrimeEn,PrimeNuclN,ResponseMuN,BundleEnReg,EventWeight,ev_id,cluster_id,PulsesChID,PulsesAmpl,PulsesTime,PulsesFlg,is_signal,mu_local_id,X,Y,Z,Xrel,Yrel,Zrel,num_signal_hits,num_signal_strings,t_res,RespMuTheta,RespMuPhi,RespMuTrackX,RespMuTrackY,RespMuTrackZ,RespMuDelay,RespMuEn,mu_local_idfor_mu
i32,f32,f32,f32,i32,i32,f32,f32,str,i8,list[i16],list[f32],list[f32],list[i32],list[bool],list[i16],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],i16,i8,f64,list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[i16]
144,142.48761,205.069519,264.505249,402,12,11177.198242,1.0,"""muatm_10170_0""",2,"[769, 745, … 742]","[0.453675, 0.487854, … 1.244507]","[-2418.305664, -2407.599609, … 2391.425293]","[0, 0, … 0]","[false, false, … false]","[-1, -1, … -1]","[-103.789627, -76.687614, … -76.687614]","[284.666687, 240.057663, … 240.057663]","[-66.488144, 113.498863, … 68.478867]","[-58.110868, -31.008856, … -31.008856]","[-4.861609, -49.470633, … -49.470633]","[-67.601007, 112.386, … 67.366003]",19,5,-144.764939,"[142.48761, 142.48761, … 142.48761]","[205.069519, 205.069519, … 205.069519]","[-185.562256, -140.511734, … -193.940186]","[273.968506, 259.521942, … 239.393097]","[39.915855, 13.289016, … 56.987972]","[0.0, 0.0, … 0.0]","[106.167679, 25.54468, … 992.909485]","[0, 1, … 11]"
144,142.48761,205.069519,264.505249,402,12,11177.198242,1.0,"""muatm_10170_0""",6,"[1785, 1979, … 1905]","[1.322611, 0.339449, … 0.553845]","[-2271.547607, -2126.994141, … 2483.211426]","[0, 0, … 0]","[false, false, … false]","[-1, -1, … -1]","[-222.545593, -313.487579, … -305.115631]","[177.842667, 191.241669, … 97.512672]","[53.693867, 264.288818, … 234.093903]","[49.316767, -41.625219, … -33.253271]","[30.902242, 44.301244, … -49.427752]","[52.505432, 263.100383, … 232.905468]",12,4,-65.124708,"[142.48761, 142.48761, … 142.48761]","[205.069519, 205.069519, … 205.069519]","[-185.562256, -140.511734, … -193.940186]","[273.968506, 259.521942, … 239.393097]","[39.915855, 13.289016, … 56.987972]","[0.0, 0.0, … 0.0]","[106.167679, 25.54468, … 992.909485]","[0, 1, … 11]"
75,131.766022,169.785034,164.788055,402,8,6046.333496,1.0,"""muatm_10170_4""",4,"[1394, 1247, … 1220]","[2.583993, 0.963218, … 0.938019]","[-2321.302734, -2285.292236, … 2650.441895]","[0, 0, … 0]","[false, false, … false]","[-1, -1, … -1]","[20.216366, 115.170372, … 113.600372]","[-258.987305, -321.128326, … -270.246277]","[126.630859, 81.320854, … 216.261871]","[-40.992133, 53.961874, … 52.391874]","[42.604509, -19.536512, … 31.345537]","[127.881517, 82.571512, … 217.512529]",10,4,-213.691004,"[131.766022, 131.766022, … 131.766022]","[169.785034, 169.785034, … 169.785034]","[-97.941811, -115.418037, … -91.434341]","[-235.658493, -229.785095, … -235.184021]","[61.136772, 81.562347, … 54.059608]","[0.0, 0.0, … 0.0]","[148.718048, -16.340019, … 212.316788]","[0, 1, … 7]"
58,128.333176,201.290375,458.393524,1608,9,10194.357422,1.0,"""muatm_10170_5""",5,"[1540, 1622, … 1650]","[0.852532, 0.678877, … 0.901192]","[-2376.115234, -2319.799561, … 2541.668457]","[0, 0, … 0]","[false, false, … false]","[-1, -1, … -1]","[-197.581619, -312.965637, … -312.965637]","[-140.047302, -161.272308, … -161.272308]","[157.6409, -232.697144, … 187.985855]","[60.158501, -55.225517, … -55.225517]","[-2.030605, -23.255611, … -23.255611]","[157.661532, -232.676511, … 188.006487]",9,4,284.372641,"[128.333176, 128.333176, … 128.333176]","[201.290375, 201.290375, … 201.290375]","[-237.817795, -185.393295, … -197.386108]","[-82.297729, -59.120495, … -80.984665]","[318.035919, 245.615768, … 269.78833]","[0.0, 0.0, … 0.0]","[149.072189, 579.408508, … 220.847931]","[0, 1, … 8]"
72,132.546112,212.479462,100.837494,14,5,9990.177734,1.0,"""muatm_10170_6""",4,"[1437, 1193, … 1394]","[1.307053, 1.014979, … 0.821642]","[-2518.487793, -2227.416016, … 2323.342285]","[0, 0, … 0]","[false, false, … false]","[-1, -1, … -1]","[60.813377, 113.600372, … 20.216366]","[-303.27533, -270.2

In [22]:
df.write_parquet('./test.parquet')

In [23]:
df: pd.DataFrame
df.to_hdf

PulsesN,PrimeTheta,PrimePhi,PrimeEn,PrimeNuclN,ResponseMuN,BundleEnReg,EventWeight,ev_id,cluster_id,PulsesChID,PulsesAmpl,PulsesTime,PulsesFlg,is_signal,X,Y,Z,Xrel,Yrel,Zrel,num_signal_hits,num_signal_strings,RespMuTheta,RespMuPhi,RespMuTrackX,RespMuTrackY,RespMuTrackZ,RespMuDelay,RespMuEn
i32,f32,f32,f32,i32,i32,f32,f32,str,i8,list[i16],list[f32],list[f32],list[i32],list[bool],list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],i16,i8,list[f32],list[f32],list[f32],list[f32],list[f32],list[f32],list[f32]
144,142.48761,205.069519,264.505249,402,12,11177.198242,1.0,"""muatm_10170_0""",2,"[769, 745, … 742]","[0.453675, 0.487854, … 1.244507]","[-636.141846, -625.435974, … 4173.588867]","[0, 0, … 0]","[false, false, … false]","[-103.789627, -76.687614, … -76.687614]","[284.666687, 240.057663, … 240.057663]","[-66.488144, 113.498863, … 68.478867]","[-58.110868, -31.008856, … -31.008856]","[-4.861609, -49.470633, … -49.470633]","[-67.601007, 112.386, … 67.366003]",19,5,"[142.48761, 142.48761, … 142.48761]","[205.069519, 205.069519, … 205.069519]","[-185.562256, -140.511734, … -193.940186]","[273.968506, 259.521942, … 239.393097]","[39.915855, 13.289016, … 56.987972]","[0.0, 0.0, … 0.0]","[106.167679, 25.54468, … 992.909485]"
144,142.48761,205.069519,264.505249,402,12,11177.198242,1.0,"""muatm_10170_0""",6,"[1785, 1979, … 1905]","[1.322611, 0.339449, … 0.553845]","[-489.384003, -344.830444, … 4265.375]","[0, 0, … 0]","[false, false, … false]","[-222.545593, -313.487579, … -305.115631]","[177.842667, 191.241669, … 97.512672]","[53.693867, 264.288818, … 234.093903]","[49.316767, -41.625219, … -33.253271]","[30.902242, 44.301244, … -49.427752]","[52.505432, 263.100383, … 232.905468]",12,4,"[142.48761, 142.48761, … 142.48761]","[205.069519, 205.069519, … 205.069519]","[-185.562256, -140.511734, … -193.940186]","[273.968506, 259.521942, … 239.393097]","[39.915855, 13.289016, … 56.987972]","[0.0, 0.0, … 0.0]","[106.167679, 25.54468, … 992.909485]"
75,131.766022,169.785034,164.788055,402,8,6046.333496,1.0,"""muatm_10170_4""",4,"[1394, 1247, … 1220]","[2.583993, 0.963218, … 0.938019]","[-539.138916, -503.128479, … 4432.605469]","[0, 0, … 0]","[false, false, … false]","[20.216366, 115.170372, … 113.600372]","[-258.987305, -321.128326, … -270.246277]","[126.630859, 81.320854, … 216.261871]","[-40.992133, 53.961874, … 52.391874]","[42.604509, -19.536512, … 31.345537]","[127.881517, 82.571512, … 217.512529]",10,4,"[131.766022, 131.766022, … 131.766022]","[169.785034, 169.785034, … 169.785034]","[-97.941811, -115.418037, … -91.434341]","[-235.658493, -229.785095, … -235.184021]","[61.136772, 81.562347, … 54.059608]","[0.0, 0.0, … 0.0]","[148.718048, -16.340019, … 212.316788]"
58,128.333176,201.290375,458.393524,1608,9,10194.357422,1.0,"""muatm_10170_5""",5,"[1540, 1622, … 1650]","[0.852532, 0.678877, … 0.901192]","[-593.951416, -537.635864, … 4323.832031]","[0, 0, … 0]","[false, false, … false]","[-197.581619, -312.965637, … -312.965637]","[-140.047302, -161.272308, … -161.272308]","[157.6409, -232.697144, … 187.985855]","[60.158501, -55.225517, … -55.225517]","[-2.030605, -23.255611, … -23.255611]","[157.661532, -232.676511, … 188.006487]",9,4,"[128.333176, 128.333176, … 128.333176]","[201.290375, 201.290375, … 201.290375]","[-237.817795, -185.393295, … -197.386108]","[-82.297729, -59.120495, … -80.984665]","[318.035919, 245.615768, … 269.78833]","[0.0, 0.0, … 0.0]","[149.072189, 579.408508, … 220.847931]"
72,132.546112,212.479462,100.837494,14,5,9990.177734,1.0,"""muatm_10170_6""",4,"[1437, 1193, … 1394]","[1.307053, 1.014979, … 0.821642]","[-736.324097, -445.252319, … 4105.505859]","[0, 0, … 0]","[false, false, … false]","[60.813377, 113.600372, … 20.216366]","[-303.27533, -270.246277, … -258.987305]","[231.318863, -189.075134, … 126.630859]","[-0.395121, 52.391874, … -40.992133]","[-1.683516, 31.345537, … 42.604509]","[232.569521, -187.824476, … 127.881517]",12,4,"[132.546112, 132.546112, … 132.546112]","[212.479462, 212.479462, … 212.

In [48]:
df_load = pl.read_parquet('./test.parquet', n_rows=10_000, row_index_name='#', row_index_offset=10_001)

In [ ]:
import pyarrow.parquet as pq

parquet_file = pq.ParquetFile('./test.parquet')

for batch in parquet_file.iter_batches(1024):
    print("RecordBatch")
    t0=time()
    batch_load = pl.DataFrame(batch)
    train_arr = batch_load['PulsesAmpl', 'PulsesTime', 'Xrel', 'Yrel', 'Zrel']
    train_arr = train_arr.to_numpy()
    print(f"Loading_time: {time()-t0:.4f} s")
    break

In [ ]:
batch_load

In [ ]:
train_arr[100,0].shape

In [6]:
events, pulses, muons = dfs[0:3]

In [40]:
df_pulses_flat = pulses.explode([c for c in pulses.columns if c not in ['ev_id']])

In [ ]:
df_pulses_flat = df_pulses_flat.with_columns(
            is_signal=(pl.col('PulsesFlg')!=0),
            cluster_id=(pl.col('PulsesChID')//288).cast(pl.Int8),
            string_id=(pl.col('PulsesChID')//36)
            )

In [ ]:
df_pulses_flat

In [67]:
df_pulses_enr = df_pulses_flat.join(
        proc.OM_coords[['PulsesChID', 'Xrel','Yrel','Zrel']], on='PulsesChID', how='left'
    ).sort(
        ['ev_id', 'PulsesTime']
    )

query = [
        pl.col(c_name) for c_name in df_pulses_enr.columns 
        if c_name not in ['ev_id', 'cluster_id', 'string_id']
    ] + [
        pl.col('is_signal').alias('num_signal_hits').sum().cast(pl.Int16),
        pl.col('string_id').filter(pl.col('is_signal')).alias('num_signal_strings').n_unique().cast(pl.Int8)
    ]
df_out = df_pulses_enr.group_by(
        ['ev_id', 'cluster_id'], maintain_order=True
    ).agg(query)


In [ ]:
df_out

In [ ]:
len(df_out['num_signal_strings'][2])

In [ ]:
pulses.memory_usage()/1024/1024

In [ ]:
muons

# Test Filtering

In [12]:
events_f, pulses_f, muons_f = proc.enrich_and_filter(events, pulses, muons)

In [ ]:
pulses_f

# Invent Saving

In [ ]:
pulses_f_f = pulses_f[['PulsesChID', 'PulsesAmpl',	'PulsesTime', 'PulsesFlg']]
pulses_f_f.memory_usage()/1024/1024, pulses_f_f.dtypes

In [ ]:
pulses_f_f.dtypes

In [ ]:
#pd.concat([pulses_f]*10).to_hdf('pulses_test.h5', key='pulses_f', mode='w', complevel=9)

0.192/10*20_000*1.3

In [25]:
#pd.concat([pulses_f]*10).to_hdf('events_test.h5', key='pulses_f', mode='w', complevel=9)

In [ ]:
pulses_f

In [67]:
pulses_fn = pulses_f.convert_dtypes()

In [ ]:
pulses_f.index

In [ ]:
#TODO fix dtypes in reader

In [ ]:
pulses_f.dtypes, pulses_fn.dtypes

In [ ]:
pulses_fn.memory_usage()/1024/1024

In [ ]:
pulses_f.memory_usage()/1024/1024

In [ ]:
pulses_f_f.to_hdf('pulses_test.h5', key='pulses_f', mode='w', complevel=1)

In [24]:
muons_f.to_hdf('muons_f.h5', key='pulses_f', mode='w', complevel=1)

In [ ]:
pd.read_hdf('/home/albert/Baikal-ML/ipynb_tests/pulses_test.h5')#, start=0, stop=100_000)#, key='pulses_f')

In [ ]:
np.int16(8000), np.array(['muatm_1000_12'], dtype=type(b'str'))

In [ ]:
sys.getsizeof('muatm_1000_12_6'), sys.getsizeof(b'muatm_1000_12_6')

In [ ]:
import h5py as h5

with h5.File("/home/albert/Baikal-ML/ipynb_tests/muons_f.h5") as hf:
   #print(hf['pulses_f'].keys())
    print(hf['pulses_f/block0_items'][:])

# Invent Filtering

In [13]:
signal_hits_per_cluster = pulses[['is_signal']].groupby(['ev_id','cluster_id']).sum().rename(columns={'is_signal': 'signal_hits_num'})

In [ ]:
signal_hits_per_cluster

In [15]:
string_nums = pulses[pulses['is_signal']][['string_id']].groupby(['ev_id','cluster_id']
    ).nunique(
    ).rename(columns={'string_id': 'unique_signal_strings_num'})

In [16]:
events_enrich = events.join(signal_hits_per_cluster, how='left').join(string_nums, how='left')
events_filter = events_enrich[(events_enrich['signal_hits_num']>=5) & (events_enrich['unique_signal_strings_num']>=2)]

In [ ]:
events_filter

In [ ]:
bad_idxs = signal_hits_per_cluster.index[signal_hits_per_cluster['signal_hits_num']<10000].union(string_nums.index[string_nums['unique_signal_strings_num']<2])
pulses_filter = pulses.drop(bad_idxs)

In [ ]:
pulses_filter[pulses_filter['is_signal']]

In [20]:
muons_filter = muons.loc[events_filter.reset_index()['ev_id']]

In [ ]:
muons_filter